In [ ]:
from pathlib import Path
#%matplotlib notebook
from core.triangulation_calibration_module import Calibration, Triangulation_Positions, Triangulation_Recordings

In [ ]:
a = Calibration(calibration_directory = Path('test_data/220826_Calibration/'), 
                project_config_filepath = Path('test_data/project_config.yaml'),
                recording_config_filepath = Path('test_data/recording_config.yaml'),
                overwrite = False, load_calibration = True, max_frame_count = 300, use_gpu = False, output_directory = Path('test_data/220826_Calibration/'))

In [ ]:
a.synchronization_crossvalidation.plot(save_to_disk = False)

In [ ]:
for individual_plot in a.synchronization_individuals:
    individual_plot.plot(save_to_disk = False)
for individual_plot in a.led_detection_individuals:
    individual_plot.plot(save_to_disk = False)

In [ ]:
a.run_calibration(verbose = True)

In [ ]:
b = Triangulation_Recordings(recording_directory = Path('test_data/220826_Recording/'), 
                             calibration_toml_filepath = Path('test_data/220826_Calibration/calibration_220826.toml'), 
                             recording_config_filepath = Path('test_data/recording_config.yaml'), 
                             project_config_filepath=Path.cwd().joinpath('test_data/project_config.yaml'),
                             output_directory = Path("test_data/220826_Recording/"), overwrite = False, load_calibration=True, use_gpu = False)

In [ ]:
b.run_triangulation()

In [ ]:
b.df

In [ ]:
# distinguish between max_frame_count! (used in intrinsic_calibrations and save_videoto_disk)

## dependencies

In [ ]:
"""
pip install aniposelib
pip install ffmpeg
pip install ffmpeg-python
pip install imageio-ffmpeg
jupyter lab
"""